In [3]:
from datasets import load_dataset
from torchvision.transforms import Compose, RandomResizedCrop, RandomHorizontalFlip, Resize, CenterCrop, ToTensor, Normalize

dataset = load_dataset("imagefolder", data_dir="D:/Study/imagenet_mini/imagenet-mini/")
train_ds = dataset["train"]
val_ds = dataset["test"]

Resolving data files:   0%|          | 0/34745 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/3923 [00:00<?, ?it/s]

In [4]:
from transformers import AutoImageProcessor

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50", cache_dir = "D:/Study/microsoft/")
size = processor.size["shortest_edge"]

normalize = Normalize(mean=processor.image_mean, std=processor.image_std)

train_transforms = Compose([
    RandomResizedCrop(size),
    RandomHorizontalFlip(),
    ToTensor(),
    normalize,
])

val_transforms = Compose([
    Resize(256),
    CenterCrop(size),
    ToTensor(),
    normalize,
])

# 应用预处理函数
def preprocess_train(examples):
    examples['pixel_values'] = [train_transforms(img.convert('RGB')) for img in examples['image']]
    return examples

def preprocess_val(examples):
    examples['pixel_values'] = [val_transforms(img.convert('RGB')) for img in examples['image']]
    return examples

train_dataset = train_ds.map(preprocess_train, batched=True)
val_dataset = val_ds.map(preprocess_val, batched=True)

train_dataset.set_format("torch", columns=["pixel_values", "label"])
val_dataset.set_format("torch", columns=["pixel_values", "label"])

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Map:   0%|          | 0/34745 [00:00<?, ? examples/s]

D:\miniforge3\envs\torch\lib\site-packages\PIL\TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Map:   0%|          | 0/3923 [00:00<?, ? examples/s]

In [5]:
from transformers import ResNetForImageClassification

In [25]:
# 加载模型
model = ResNetForImageClassification.from_pretrained(
    "microsoft/resnet-18",
    cache_dir = "D:/Study/microsoft/",
    num_labels=1000,
    ignore_mismatched_sizes=True
)

# 冻结所有层（默认情况下参数是requires_grad=True）
for param in model.parameters():
    param.requires_grad = False

# # 仅解冻分类头（最后一层）
for param in model.classifier.parameters():
    param.requires_grad = True

# 或者解冻最后几个阶段（例如stage4）
# for name, param in model.named_parameters():
#     if "stages.3" in name:  # 根据ResNet结构选择解冻的层
#         param.requires_grad = True

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

D:\miniforge3\envs\torch\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\Study\microsoft\models--microsoft--resnet-18. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [26]:
for name, layer in model.named_children():
    print(f"Layer: {name} | Type: {type(layer)}")

Layer: resnet | Type: <class 'transformers.models.resnet.modeling_resnet.ResNetModel'>
Layer: classifier | Type: <class 'torch.nn.modules.container.Sequential'>


In [27]:
from torch.utils.data import DataLoader

# 设置批量大小（根据GPU显存调整）
batch_size = 32

train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=batch_size,
    pin_memory=True,  # 加速数据加载
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    pin_memory=True,
)

In [28]:
import torch
from torch import nn
from torch.optim import AdamW

# 交叉熵损失函数
loss_fn = nn.CrossEntropyLoss()

# 筛选需要训练的参数
trainable_params = [p for p in model.parameters() if p.requires_grad]

# 使用AdamW优化器，学习率根据任务调整
optimizer = AdamW(trainable_params, lr=1e-4, weight_decay=1e-5)

In [32]:
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 10  # 根据需求调整
best_val_acc = 0.0
total_time = 0.0

for epoch in range(epochs):
    start_time = time.time()
    # 训练阶段
    model.train()
    train_loss = 0.0
    for batch in train_loader:
        inputs = batch["pixel_values"].to(device)
        labels = batch["label"].to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    # 验证阶段
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs = batch["pixel_values"].to(device)
            labels = batch["label"].to(device)
            
            outputs = model(inputs)
            loss = loss_fn(outputs.logits, labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_time = time.time() - start_time
    total_time += epoch_time
    
    # 计算指标
    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    val_acc = correct / total
    
    print(
        f'Epoch {epoch+1}/{epochs}, '
        f'Train Loss: {avg_train_loss:.4f}, '
        f'Val Loss: {avg_val_loss:.4f}, '
        f'Val Acc: {val_acc:.4f}, '
        f'Epoch Time: {epoch_time:.2f}s, '
        f'Total Time: {total_time:.2f}s'
    )
    # 保存最佳模型
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("Saved best model!")

Epoch 1/10, Train Loss: 1.3100, Val Loss: 1.2795, Val Acc: 0.6882, Epoch Time: 854.44s, Total Time: 854.44s
Saved best model!
Epoch 2/10, Train Loss: 1.2582, Val Loss: 1.2889, Val Acc: 0.6857, Epoch Time: 857.82s, Total Time: 1712.26s
Epoch 3/10, Train Loss: 1.2153, Val Loss: 1.2902, Val Acc: 0.6865, Epoch Time: 846.43s, Total Time: 2558.69s
Epoch 4/10, Train Loss: 1.1736, Val Loss: 1.2955, Val Acc: 0.6852, Epoch Time: 856.01s, Total Time: 3414.70s
Epoch 5/10, Train Loss: 1.1391, Val Loss: 1.3064, Val Acc: 0.6816, Epoch Time: 846.54s, Total Time: 4261.25s
Epoch 6/10, Train Loss: 1.1016, Val Loss: 1.3100, Val Acc: 0.6814, Epoch Time: 848.04s, Total Time: 5109.29s
Epoch 7/10, Train Loss: 1.0680, Val Loss: 1.3218, Val Acc: 0.6778, Epoch Time: 847.56s, Total Time: 5956.85s
Epoch 8/10, Train Loss: 1.0301, Val Loss: 1.3261, Val Acc: 0.6768, Epoch Time: 858.42s, Total Time: 6815.27s
Epoch 9/10, Train Loss: 0.9996, Val Loss: 1.3394, Val Acc: 0.6775, Epoch Time: 879.45s, Total Time: 7694.72s
Ep